In [21]:
from skimage import io
from skimage.morphology import skeletonize
import numpy as np
import matplotlib.pyplot as plt
import os
import json

def contar_vecinos(x, y, imagen):
    kernel = imagen[x-1:x+2, y-1:y+2]
    return np.sum(kernel) - imagen[x, y]

def clasificar_nodos(esqueleto):
    rows, cols = esqueleto.shape
    extremos = []
    bifurcaciones = []
    trifurcaciones = []
    intermedios = []

    for x in range(1, rows-1):
        for y in range(1, cols-1):
            if esqueleto[x, y]:
                vecinos = contar_vecinos(x, y, esqueleto)
                if vecinos == 1:
                    extremos.append([x, y])
                elif vecinos == 2:
                    intermedios.append([x, y])
                elif vecinos == 3:
                    bifurcaciones.append([x, y])
                elif vecinos >= 4:
                    trifurcaciones.append([x, y])
    
    return extremos, bifurcaciones, trifurcaciones, intermedios

def find_intermediate_nodes(esqueleto, extremos, bifurcaciones, trifurcaciones):
    intermedios = []
    for x in range(1, esqueleto.shape[0] - 1):
        for y in range(1, esqueleto.shape[1] - 1):
            if esqueleto[x, y]:
                vecinos = esqueleto[x-1:x+2, y-1:y+2].sum() - 1
                if vecinos == 2 and [x, y] not in extremos and [x, y] not in bifurcaciones and [x, y] not in trifurcaciones:
                    intermedios.append([x, y])
    intermedios.sort(key=lambda node: node[0])
    mod = 25
    intermedios = intermedios[::mod]
    return intermedios

def eliminar_solapamientos(nodos):
    nodos_filtrados = []
    for nodo in nodos:
        solapamiento = any(np.linalg.norm(np.array(nodo) - np.array(otro_nodo)) < 2 for otro_nodo in nodos_filtrados)
        if not solapamiento:
            nodos_filtrados.append(nodo)
    return nodos_filtrados

def guardar_imagen_con_nodos(esqueleto_color, extremos, bifurcaciones, trifurcaciones, intermedios, archivo_destino):
    fig, ax = plt.subplots()
    ax.imshow(esqueleto_color, interpolation='nearest')
    # Eliminar solapamientos antes de dibujar
    todos_los_nodos = extremos + bifurcaciones + trifurcaciones
    todos_los_nodos = eliminar_solapamientos(todos_los_nodos)
    for x, y in todos_los_nodos:
        if [x, y] in extremos:
            ax.plot(y, x, 'go')  # Verde para extremos
        elif [x, y] in bifurcaciones:
            ax.plot(y, x, 'ro')  # Rojo para bifurcaciones
        elif [x, y] in trifurcaciones:
            ax.plot(y, x, 'bo')  # Azul para trifurcaciones
    for x, y in intermedios:
        ax.plot(y, x, 'o', color='grey', markersize=5, alpha=0.5)  # Gris para nodos intermedios
    ax.axis('off')
    plt.savefig(archivo_destino, bbox_inches='tight')
    plt.close(fig)

def generar_archivo_json(extremos, bifurcaciones, trifurcaciones, intermedios, archivo_destino):
    estructura_grafo = {
        "nodos": {
            "extremos": extremos,
            "bifurcaciones": bifurcaciones,
            "trifurcaciones": trifurcaciones,
            "intermedios": intermedios
        }
    }
    with open(archivo_destino, 'w') as file:
        json.dump(estructura_grafo, file, indent=4)

def procesar_imagenes_en_carpeta(carpeta, carpeta_destino):
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    archivos = os.listdir(carpeta)
    archivos_pgm = [archivo for archivo in archivos if archivo.endswith('_gt.pgm')]
    
    for archivo in archivos_pgm:
        image_path = os.path.join(carpeta, archivo)
        image = io.imread(image_path)
        
        esqueleto = skeletonize(image > 0)
        
        extremos, bifurcaciones, trifurcaciones, _ = clasificar_nodos(esqueleto)
        intermedios = find_intermediate_nodes(esqueleto, extremos, bifurcaciones, trifurcaciones)
        
        esqueleto_color = np.zeros((*esqueleto.shape, 3), dtype=np.uint8)
        esqueleto_color[esqueleto] = [255, 255, 0]  # Amarillo
        
        nombre_archivo_imagen = archivo.replace('_gt.pgm', '_esqueleto.png')
        archivo_imagen_destino = os.path.join(carpeta_destino, nombre_archivo_imagen)
        guardar_imagen_con_nodos(esqueleto_color, extremos, bifurcaciones, trifurcaciones, intermedios, archivo_imagen_destino)
        
        nombre_archivo_json = archivo.replace('_gt.pgm', '_grafo.json')
        archivo_json_destino = os.path.join(carpeta_destino, nombre_archivo_json)
        generar_archivo_json(extremos, bifurcaciones, trifurcaciones, intermedios, archivo_json_destino)
        
        print(f'Archivo JSON generado: {archivo_json_destino}')
        print(f'Imagen esqueletizada guardada: {archivo_imagen_destino}')

carpeta_imagenes = './database'
carpeta_destino = './database_problema2'
procesar_imagenes_en_carpeta(carpeta_imagenes, carpeta_destino)


Archivo JSON generado: ./database_problema2\10_grafo.json
Imagen esqueletizada guardada: ./database_problema2\10_esqueleto.png
Archivo JSON generado: ./database_problema2\11_grafo.json
Imagen esqueletizada guardada: ./database_problema2\11_esqueleto.png
Archivo JSON generado: ./database_problema2\12_grafo.json
Imagen esqueletizada guardada: ./database_problema2\12_esqueleto.png
Archivo JSON generado: ./database_problema2\13_grafo.json
Imagen esqueletizada guardada: ./database_problema2\13_esqueleto.png
Archivo JSON generado: ./database_problema2\14_grafo.json
Imagen esqueletizada guardada: ./database_problema2\14_esqueleto.png
Archivo JSON generado: ./database_problema2\15_grafo.json
Imagen esqueletizada guardada: ./database_problema2\15_esqueleto.png
Archivo JSON generado: ./database_problema2\16_grafo.json
Imagen esqueletizada guardada: ./database_problema2\16_esqueleto.png
Archivo JSON generado: ./database_problema2\17_grafo.json
Imagen esqueletizada guardada: ./database_problema2\1